# This is a folk of [this notebook](https://www.kaggle.com/heyytanay/0-9-auc-pytorch-training-amp-vit-kfolds).
I omitted AMP (Automatic Mixed Precision), and found trainng speed becomes faster (2.18it/s -> 2.30it/s). Do you know why?

<h1 align='center' style='background: #1eeacf'>1. Imports and Data Loading</h1>

In [ ]:
import sys
sys.path.append("../input/timmeffnetv2")

In [ ]:
import platform
import numpy as np
import pandas as pd
from tqdm.notebook import tqdm
import cv2
import gc
import matplotlib.pyplot as plt
from sklearn.metrics import roc_auc_score
from sklearn.model_selection import StratifiedKFold

import torch
import timm
import torch.nn as nn
import torch.nn.functional as F
from torch.cuda.amp import GradScaler, autocast
from torch.utils.data import Dataset, DataLoader

import warnings
warnings.simplefilter('ignore')

In [ ]:
train_labels = pd.read_csv('../input/seti-breakthrough-listen/train_labels.csv')
train_labels['path'] = train_labels['id'].apply(lambda x: f'../input/seti-breakthrough-listen/train/{x[0]}/{x}.npy')
train_labels.head()

In [ ]:
def plotFromFoldScores(score_dict, colors=None):
    if colors is None:
        colors = ['r', 'g', 'y', 'b', 'm']
    
    plt.figure(figsize=(10, 7))
    for fold_num, scores in score_dict.items():
        plt.plot(scores, f'{colors[fold_num]}o-', label=f'Fold {fold_num}')
    
    plt.ylabel("Validation ROC-AUC Score")
    plt.xlabel("Epochs")
    plt.title("Val ROC-AUC Scores in all Folds")
    plt.legend()
    plt.show()

In [ ]:
class Config:
    N_SPLITS = 5
    model_name = 'vit_base_patch16_224'
    resize = (224, 224)
    TRAIN_BS = 32
    VALID_BS = 16
    num_workers = 8
    NB_EPOCHS = 5
    scaler = GradScaler()

<h1 align='center' style='background: #1eeacf'>2. Trainer Class - Training and Validation Code</h1>

In [ ]:
class Trainer:
    def __init__(self, model, optimizer, scheduler, train_dataloader, valid_dataloader, device):
        self.model = model
        self.optimizer = optimizer
        self.scheduler = scheduler
        self.train_data = train_dataloader
        self.valid_data = valid_dataloader
        self.loss_fn = self.yield_loss
        self.val_loss_fn = self.yield_loss
        self.device = device
        
    def yield_loss(self, outputs, targets):
        return nn.BCEWithLogitsLoss()(outputs, targets)
    
    def train_one_epoch(self):
        prog_bar = tqdm(enumerate(self.train_data), total=len(self.train_data))
        self.model.train()
        avg_loss = 0
        #with autocast():
        for idx, inputs in prog_bar:
            image = inputs[0].to(self.device, dtype=torch.float)
            targets = inputs[1].to(self.device, dtype=torch.float)

            outputs = self.model(image)

            loss = self.loss_fn(outputs, targets.view(-1, 1))
            prog_bar.set_description('loss: {:.2f}'.format(loss.item()))

            loss.backward()
            self.optimizer.step()
            #Config.scaler.scale(loss).backward()
            #Config.scaler.step(self.optimizer)
            #Config.scaler.update()
            self.optimizer.zero_grad(set_to_none=True)

            avg_loss += loss.item()
                
        return avg_loss / len(self.train_data)
    
    def valid_one_epoch(self):
        prog_bar = tqdm(enumerate(self.valid_data), total=len(self.valid_data))
        self.model.eval()
        all_targets = []
        all_predictions = []
        avg_loss = 0
        with torch.no_grad():
            for idx, inputs in prog_bar:
                image = inputs[0].to(self.device, dtype=torch.float)
                targets = inputs[1].to(self.device, dtype=torch.float)

                outputs = self.model(image)
                
                val_loss = self.val_loss_fn(outputs, targets.view(-1, 1))
                prog_bar.set_description('val_loss: {:.2f}'.format(val_loss.item()))
                
                all_targets.extend(targets.cpu().detach().numpy().tolist())
                all_predictions.extend(torch.sigmoid(outputs).cpu().detach().numpy().tolist())
                
                avg_loss += val_loss.item()
        val_roc_auc = roc_auc_score(all_targets, all_predictions)
        return val_roc_auc, avg_loss / len(self.valid_data)
    
    def get_model(self):
        return self.model

<h1 align='center' style='background: #1eeacf'>3. Custom Dataset - SETIData</h1>

In [ ]:
class SETIData(Dataset):
    def __init__(self, images, targets, is_test=False, augmentations=None): 
        self.images = images
        self.targets = targets
        self.is_test = is_test
        self.augmentations = augmentations
        
    def __getitem__(self, index):
        image = np.load(self.images[index]).astype(np.float32)
        image = np.vstack(image).transpose((1, 0))
        image = cv2.resize(image, dsize=Config.resize, interpolation=cv2.INTER_CUBIC)
        
        if self.augmentations is not None:
            augmented = self.augmentations(image=image)
            image = augmented["image"]
        else:
            image = image[np.newaxis, :, :]
            
        if self.is_test:
            return image
        
        else:
            target = self.targets[index]
            return image, target
    
    def __len__(self):
        return len(self.images)

<h1 align='center' style='background: #1eeacf'>4. Model Classes - ViT and EffNetV2</h1>

In [ ]:
print("You can see the available models from EfficientNetV2 family here:")
timm.list_models('*vit*_224*')

In [ ]:
class EffNetV2(nn.Module):
    def __init__(self, pretrained=True) -> None:
        super(EffNetV2, self).__init__()
        self.backbone = timm.create_model(Config.model_name, pretrained=pretrained, in_chans=1)
        self.backbone.classifier = nn.Linear(self.backbone.classifier.in_features, 1)
        
    def forward(self, x) -> torch.Tensor:
        out = self.backbone(x)
        return out
    
class VITModel(nn.Module):
    """
    Model Class for VIT Model
    """
    def __init__(self, model_name=Config.model_name, pretrained=True):
        super(VITModel, self).__init__()
        self.model = timm.create_model(model_name, pretrained, in_chans=1)
        self.model.head = nn.Linear(self.model.head.in_features, 1)
    
    def forward(self, x):
        x = self.model(x)
        return x

<h1 align='center' style='background: #1eeacf'>5. Main Training Code</h1>

In [ ]:
# Training Code
if __name__ == '__main__':
    if torch.cuda.is_available():
        print("[INFO] Using GPU: {}\n".format(torch.cuda.get_device_name()))
        DEVICE = torch.device('cuda:0')
    else:
        print("\n[INFO] GPU not found. Using CPU: {}\n".format(platform.processor()))
        DEVICE = torch.device('cpu')
    
    kfold = StratifiedKFold(n_splits=Config.N_SPLITS, shuffle=True, random_state=2021)
    
    fold_scores = {}
    
    for fold_, (trn_idx, val_idx) in enumerate(kfold.split(train_labels, train_labels['target'])):
        print(f"{'='*40} Fold: {fold_} {'='*40}")
        
        train_data = train_labels.loc[trn_idx]
        valid_data = train_labels.loc[val_idx]
        
        print(f"[INFO] Training on {trn_idx.shape[0]} samples and validating on {valid_data.shape[0]} samples")

        # Make Training and Validation Datasets
        training_set = SETIData(
            images=train_data['path'].values,
            targets=train_data['target'].values
        )

        validation_set = SETIData(
            images=valid_data['path'].values,
            targets=valid_data['target'].values
        )

        train = DataLoader(
            training_set,
            batch_size=Config.TRAIN_BS,
            shuffle=True,
            num_workers=8,
            pin_memory=True
        )

        valid = DataLoader(
            validation_set,
            batch_size=Config.VALID_BS,
            shuffle=False,
            num_workers=8
        )

        model = VITModel().to(DEVICE)
        print(f"[INFO] Training Model: {Config.model_name}")
        nb_train_steps = int(len(train_data) / Config.TRAIN_BS * Config.NB_EPOCHS)
        optimizer = torch.optim.Adam(model.parameters(), lr=1e-5)

        trainer = Trainer(model, optimizer, None, train, valid, DEVICE)
        
        per_fold_score = []
        for epoch in range(1, Config.NB_EPOCHS+1):
            print(f"\n{'--'*5} EPOCH: {epoch} {'--'*5}\n")

            # Train for 1 epoch
            tr_lss = trainer.train_one_epoch()
            
            # Validate for 1 epoch
            current_roc, vl_lss = trainer.valid_one_epoch()
            print(f"Validation ROC-AUC: {current_roc:.4f}")
            
            per_fold_score.append(current_roc)
            torch.save(trainer.get_model().state_dict(), f"{Config.model_name}_fold_{fold_}.pt")
        
        fold_scores[fold_] = per_fold_score
        del training_set, validation_set, train, valid, model, optimizer, trainer, current_roc
        gc.collect()
        torch.cuda.empty_cache()

In [ ]:
# Plot validation roc-auc scores for all 5 epochs from all 5 folds
plotFromFoldScores(fold_scores)